In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import files
import os

# Install dependencies if needed (usually pre-installed in Colab)
try:
    import pandas, numpy, scipy, seaborn, matplotlib
except ImportError:
    !pip install pandas numpy scipy seaborn matplotlib

# Upload datasets
print("Please upload fear_greed_index.csv and historical_data.csv from your Downloads folder.")
uploaded = files.upload()

# Load datasets
try:
    sentiment_df = pd.read_csv('fear_greed_index.csv')
    trader_df = pd.read_csv('historical_data.csv')
except FileNotFoundError as e:
    print(f"Error: File not uploaded correctly. Please ensure both files are uploaded: {e}")
    exit()

# Print column names for debugging
print("Sentiment Dataset Columns:", sentiment_df.columns.tolist())
print("Trader Dataset Columns:", trader_df.columns.tolist())

# Data Preparation
# Convert dates to datetime
try:
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
except KeyError:
    print("Error: 'date' column not found in sentiment dataset. Available columns:", sentiment_df.columns.tolist())
    print("Please check the date column name in fear_greed_index.csv and update the script.")
    exit()

try:
    trader_df['Date'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
except KeyError:
    print("Error: 'Timestamp IST' column not found in trader dataset. Available columns:", trader_df.columns.tolist())
    exit()

# Extract date for merging
trader_df['Date'] = trader_df['Date'].dt.date
sentiment_df['Date'] = sentiment_df['date'].dt.date

# Merge datasets on date
merged_df = pd.merge(trader_df, sentiment_df[['Date', 'value', 'classification']], on='Date', how='left')

# Filter for closed trades (non-zero PnL)
closed_trades = merged_df[merged_df['Closed PnL'] != 0].copy()

# Handle missing sentiment data
closed_trades.dropna(subset=['classification'], inplace=True)

# Performance Metrics by Sentiment
metrics = closed_trades.groupby('classification').agg({
    'Closed PnL': ['count', 'sum', 'mean', lambda x: (x > 0).mean() * 100],
    'Fee': 'mean'
}).round(2)
metrics.columns = ['Num Trades', 'Total PnL (USD)', 'Avg PnL (USD)', 'Win Rate (%)', 'Avg Fee (USD)']

# Compute Sharpe Ratio
def compute_sharpe(pnl_series):
    if len(pnl_series) < 2 or pnl_series.std() == 0:
        return np.nan
    return round(pnl_series.mean() / pnl_series.std() * np.sqrt(365), 2)

sharpe_ratios = closed_trades.groupby('classification')['Closed PnL'].apply(compute_sharpe)
metrics['Sharpe Ratio'] = sharpe_ratios

# Bar Chart: Total PnL by Sentiment
plt.figure(figsize=(8, 6))
sns.barplot(x='classification', y='Total PnL (USD)', data=metrics.reset_index(), palette='viridis')
plt.title('Total PnL by Sentiment Classification')
plt.xlabel('Sentiment Classification')
plt.ylabel('Total PnL (USD)')
plt.savefig('/content/pnl_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
files.download('/content/pnl_by_sentiment.png')
print("Bar chart saved and downloaded as 'pnl_by_sentiment.png'")

# Trader Archetypes
archetypes = closed_trades.groupby(['Account', 'classification']).agg({
    'Closed PnL': ['count', 'sum', 'mean']
}).round(2)
archetypes.columns = ['Num Trades', 'Total PnL (USD)', 'Avg PnL (USD)']
archetypes_reset = archetypes.reset_index()
archetypes_reset['Account'] = archetypes_reset['Account'].apply(lambda x: x[:6] + '...' + x[-4:])

# Bar Chart: Total PnL by Account and Sentiment
plt.figure(figsize=(10, 6))
sns.barplot(x='classification', y='Total PnL (USD)', hue='Account', data=archetypes_reset, palette='muted')
plt.title('Total PnL by Account and Sentiment')
plt.xlabel('Sentiment Classification')
plt.ylabel('Total PnL (USD)')
plt.legend(title='Account', loc='best')
plt.savefig('/content/pnl_by_account_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
files.download('/content/pnl_by_account_sentiment.png')
print("Bar chart saved and downloaded as 'pnl_by_account_sentiment.png'")

# PnL by Direction and Sentiment
side_analysis = closed_trades.groupby(['Direction', 'classification'])['Closed PnL'].agg(['sum', 'mean']).round(2)
side_analysis.columns = ['Total PnL (USD)', 'Avg PnL (USD)']
side_analysis_reset = side_analysis.reset_index()

# Bar Chart: Total PnL by Direction and Sentiment
plt.figure(figsize=(10, 6))
sns.barplot(x='classification', y='Total PnL (USD)', hue='Direction', data=side_analysis_reset, palette='deep')
plt.title('Total PnL by Direction and Sentiment')
plt.xlabel('Sentiment Classification')
plt.ylabel('Total PnL (USD)')
plt.legend(title='Direction', loc='best')
plt.savefig('/content/pnl_by_direction_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
files.download('/content/pnl_by_direction_sentiment.png')
print("Bar chart saved and downloaded as 'pnl_by_direction_sentiment.png'")

# Statistical Analysis
greed_pnl = closed_trades[closed_trades['classification'] == 'Greed']['Closed PnL']
extreme_greed_pnl = closed_trades[closed_trades['classification'] == 'Extreme Greed']['Closed PnL']
t_stat, p_value = ttest_ind(greed_pnl, extreme_greed_pnl, equal_var=False)
print(f"\nT-Test (Greed vs. Extreme Greed PnL): t={t_stat:.2f}, p={p_value:.4f}")

corr = closed_trades[['Closed PnL', 'value']].corr().iloc[0, 1]
print(f"Correlation between Sentiment Value and Closed PnL: {corr:.2f}")

# Trading Strategy Simulation
def sentiment_strategy(index_value):
    if index_value > 75:
        return "Sell"
    elif index_value < 50:
        return "Buy"
    else:
        return "Hold"

# Recent Sentiment (Last 30 Days, up to Oct 22, 2025)
recent_sentiment = sentiment_df[sentiment_df['date'] >= '2025-09-23'][['date', 'value', 'classification']].copy()
recent_sentiment['Strategy'] = recent_sentiment['value'].apply(sentiment_strategy)
recent_sentiment['Date'] = recent_sentiment['date'].dt.strftime('%Y-%m-%d')

# Bar Chart: Sentiment Values Over Last 30 Days
plt.figure(figsize=(12, 6))
sns.barplot(x='Date', y='value', hue='classification', data=recent_sentiment, palette='Set2')
plt.title('Sentiment Values and Classification (Last 30 Days)')
plt.xlabel('Date')
plt.ylabel('Sentiment Value')
plt.xticks(rotation=45)
plt.legend(title='Classification', loc='best')
plt.savefig('/content/recent_sentiment.png', dpi=300, bbox_inches='tight')
plt.close()
files.download('/content/recent_sentiment.png')
print("Bar chart saved and downloaded as 'recent_sentiment.png'")

# Bar Chart: Strategy Counts
strategy_counts = recent_sentiment['Strategy'].value_counts().reset_index()
strategy_counts.columns = ['Strategy', 'Count']
plt.figure(figsize=(8, 6))
sns.barplot(x='Strategy', y='Count', data=strategy_counts, palette='Set1')
plt.title('Trading Strategy Signals (Last 30 Days)')
plt.xlabel('Strategy')
plt.ylabel('Count of Days')
plt.savefig('/content/strategy_counts.png', dpi=300, bbox_inches='tight')
plt.close()
files.download('/content/strategy_counts.png')
print("Bar chart saved and downloaded as 'strategy_counts.png'")

print("\nAll bar charts saved and downloaded from Colab.")

Please upload fear_greed_index.csv and historical_data.csv from your Downloads folder.


Saving historical_data.csv to historical_data.csv
Saving fear_greed_index.csv to fear_greed_index.csv
Sentiment Dataset Columns: ['timestamp', 'value', 'classification', 'date']
Trader Dataset Columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']


/tmp/ipython-input-1575975637.py:77: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x='classification', y='Total PnL (USD)', data=metrics.reset_index(), palette='viridis')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Bar chart saved and downloaded as 'recent_sentiment.png'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Bar chart saved and downloaded as 'strategy_counts.png'

All bar charts saved and downloaded from Colab.
